In [45]:
import json
import math
import escher
import cobra
import cobrakbase
import modelseed_escher
from cobrakbase.core import KBaseFBAModel
from modelseed_escher.core import EscherMap

In [7]:
%run ../lib/kb_escher/kb_escher_utils.py

In [1]:
params = {
    'workspace_id': 37534, 
    'column_size': '4', 
    'report_height': '800', 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'grid_maps': [
        {
            'object_ids': ['37534/91/1'], 
            'map_id': 'custom', 
            'user_map_id': '37534/38/1', 
            'model_id': '37534/90/3'
        }, 
        {
            'object_ids': ['37534/92/1'], 
            'map_id': 'FBA Checked TCA Cycle', 
            'user_map_id': '37534/88/1', 
            'model_id': '37534/89/1'
        }
    ]
}

In [35]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
api = cobrakbase.KBaseAPI(token, dev=True)

In [21]:
escher_seed = modelseed_escher.EscherManager(escher)

In [57]:
column_size = int(params['column_size'])
grid_cells = len(params['grid_maps'])
grid_size = (column_size, math.ceil(grid_cells / column_size))
grid_size

(4, 1)

In [66]:
em_list = []
alias_index = 1
maps_loaded = set()
for grid_cell in params['grid_maps']:
    maps_loaded.add(grid_cell['map_id'])
    em = build_grid_cell(grid_cell, escher_seed, api, str(alias_index))
    alias_index += 1
    em_list.append(em)

In [68]:
for em in em_list:
    em

In [71]:
#current_grid_layout =  get_shadow_grid_layout(map_data[1].canvas, build_config);
#draw_show_grid_layout(shadow_grid_layout, api);
#"iMM904;c;c;ModelSEED.Poly-beta-hydroxybutyrate Metabolism"
build_config = {
    'maps' : [
        'model;c;c;FBA Checked TCA Cycle.json'
    ],
    'x' : grid_size[0],
    'y' : grid_size[1],
}
build_config

{'maps': [], 'x': 4, 'y': 1}

In [ ]:
#build grid map
#cached maps
#cache shadow config
#cache structures

In [63]:
grid = modelseed_escher.EscherGrid()
grid_map = grid.build(em_list, grid_size)

In [ ]:
model_id = grid_block_data['model_id']

In [34]:
grid_cell['model_id']

'37534/89/1'

In [43]:
map_data.keys()

dict_keys(['layout', 'metadata'])

In [72]:
map_data = escher_seed.get_map('ModelSEED', 'ModelSEED', 'FBA Checked TCA Cycle')

In [74]:
with open('../data/html/data/map_model/base_map.json', 'w') as fh:
    fh.write(json.dumps(map_data.escher_map))

In [145]:
params = {
    'workspace_id': 37534, 
    'column_size': 1, 
    'report_height': 800, 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'grid_maps': [
        {
            'object_ids': '37534/92/1', 
            'map_id': 'Pyrimidine Utilization', 
            'user_map_id': '37534/88/1', 
            'cpd_abundance_dataset': '', 
            'gene_expression': '37534/98/2', 
            'cpd_abundance': '37534/99/1', 
            'model_id': '37534/90/3', 
            'model_alias': 'BiGG', 
            'gene_expression_dataset': ''
        }
    ]
}

In [185]:
%run ../lib/kb_escher/kb_escher_app.py

In [182]:
kb_escher = KBaseEscher(params, api, escher_seed)

{'object_ids': '37534/92/1', 'map_id': 'Pyrimidine Utilization', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/90/3', 'model_alias': 'BiGG', 'gene_expression_dataset': ''}


In [183]:
kb_escher.build()
kb_escher.export('/Users/fliu/workspace/jupyter/python3/kbase/kb_escher/data/html/data/')

In [79]:
em_list = []
for grid_cell in params['grid_maps']:
    em = build_grid_cell(grid_cell, escher_seed, api)
    em_list.append(em)
grid = modelseed_escher.EscherGrid()
grid_map = grid.build(em_list, grid_size)

In [84]:
def get_grid_cell_cpd_paint(grid_cell):
    pass
def get_grid_cell_rxn_paint(grid_cell):
    pass
def get_grid_cell_gene_paint(grid_cell):
    pass

In [139]:

        

kb_escher.grid_size

{'object_ids': '37534/92/1', 'map_id': 'Pyrimidine Utilization', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/90/3', 'model_alias': 'BiGG', 'gene_expression_dataset': ''}


(1, 1)

In [85]:
get_grid_cell_cpd_paint(grid_cell)

In [86]:
grid_cell

{'object_ids': '37534/92/1',
 'map_id': 'Pyrimidine Utilization',
 'user_map_id': '37534/88/1',
 'cpd_abundance_dataset': '',
 'gene_expression': '37534/98/2',
 'cpd_abundance': '37534/99/1',
 'model_id': '37534/90/3',
 'model_alias': 'BiGG',
 'gene_expression_dataset': ''}

In [109]:
data = {}
grid_map_rxn_paint = {}
grid_map_gene_paint = {}
grid_map_cpd_paint = {}
for grid_cell in params['grid_maps']:
    alias = None
    if 'object_ids' in grid_cell:
        ref = api.get_object_info_from_ref(grid_cell['object_ids'])
        object_data = api.get_object(ref.id, ref.workspace_id)
        #data[grid_cell['object_ids']] = cobrakbase.core.KBaseFBA(object_data)
        flux_dist = {}
        for o in fba.data['FBAReactionVariables']:
            rxn_id = o['modelreaction_ref'].split('/')[-1]
            flux_dist[rxn_id] = o['value']
        data[grid_cell['object_ids']] = flux_dist
        #add alias
        #merge with map
        in_map = set()
        print(len(in_map))
        #save to 

In [184]:
params = {
    'workspace_id': 37534, 
    'column_size': 2, 
    'report_height': 800, 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'grid_maps': [
        {
            'object_ids': '37534/91/1', 
            'map_id': 'Pyrimidine Utilization', 
            'user_map_id': '37534/88/1', 
            'cpd_abundance_dataset': '', 
            'gene_expression': '37534/98/2', 
            'cpd_abundance': '37534/99/1', 
            'model_id': '37534/90/3', 
            'model_alias': '1', 
            'gene_expression_dataset': ''
        }, 
        {
            'object_ids': '37534/91/1', 
            'map_id': 'Serine Biosynthesis', 
            'user_map_id': '37534/88/1', 
            'cpd_abundance_dataset': '', 
            'gene_expression': '37534/98/2', 
            'cpd_abundance': '37534/99/1', 
            'model_id': '37534/90/3', 
            'model_alias': '1', 
            'gene_expression_dataset': ''
        }, 
        {
            'object_ids': '37534/92/1', 
            'map_id': 'Serine Biosynthesis', 
            'user_map_id': '37534/88/1', 
            'cpd_abundance_dataset': '', 
            'gene_expression': '37534/98/2',
            'cpd_abundance': '37534/99/1', 
            'model_id': '37534/89/1', 
            'model_alias': '1', 
            'gene_expression_dataset': ''
        }, 
        {
            'object_ids': '37534/92/1', 
            'map_id': 
            'Pyrimidine Utilization', 
            'user_map_id': '37534/88/1', 
            'cpd_abundance_dataset': '', 
            'gene_expression': '37534/98/2', 
            'cpd_abundance': '37534/99/1', 
            'model_id': '37534/89/1', 
            'model_alias': '1', 
            'gene_expression_dataset': ''
        }
    ]
}

In [186]:
kb_escher = KBaseEscher(params, api, escher_seed)

{'object_ids': '37534/91/1', 'map_id': 'Pyrimidine Utilization', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/90/3', 'model_alias': '1', 'gene_expression_dataset': ''}
{'object_ids': '37534/91/1', 'map_id': 'Serine Biosynthesis', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/90/3', 'model_alias': '1', 'gene_expression_dataset': ''}
{'object_ids': '37534/92/1', 'map_id': 'Serine Biosynthesis', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/89/1', 'model_alias': '1', 'gene_expression_dataset': ''}
{'object_ids': '37534/92/1', 'map_id': 'Pyrimidine Utilization', 'user_map_id': '37534/88/1', 'cpd_abundance_dataset': '', 'gene_expression': '37534/98/2', 'cpd_abundance': '37534/99/1', 'model_id': '37534/90/3', '

In [188]:
kb_escher.build()
kb_escher.export('/Users/fliu/workspace/jupyter/python3/kbase/kb_escher/data/html/data/')